In [1]:
import numpy as np
import cv2
import tkinter as tk
from PIL import Image, ImageTk
import random
import time


In [2]:
import random
# Función para generar un color aleatorio en el rango RGB
def generate_random_color():
    return [random.randint(0, 255) for _ in range(3)]

In [3]:
import math
# Función para calcular la distancia euclidiana entre dos colores en el espacio RGB
def euclidean_distance(color1, color2):
    return math.sqrt(sum((c1 - c2) ** 2 for c1, c2 in zip(color1, color2)))

In [4]:
# Función para generar una población inicial de colores aleatorios
def generate_initial_population(population_size):
    return [generate_random_color() for _ in range(population_size)]

In [5]:
# Función de selección de torneo
def tournament_selection(population, target_color,k=3):
    tournament_contestants = random.sample(population, k)
    return min(tournament_contestants, 
               key=lambda x: euclidean_distance(x,target_color))

In [6]:
# Función de cruce (crossover) de un punto
def single_point_crossover(parent1, parent2):
    crossover_point = random.randint(0, len(parent1) - 1)
    child = parent1[:crossover_point] + parent2[crossover_point:]
    return child


In [7]:
# Función de mutación uniforme
def mutation(individual, mutation_rate=0.8):
    mutated_individual = individual[:]
    if random.random() < mutation_rate:
        mutation_index = random.randint(0, len(individual) - 1)
        mutated_individual[mutation_index] = random.randint(0, 255)
    return mutated_individual


In [8]:
# Función para determinar si se ha alcanzado el criterio de parada
def has_converged(population, target_color, threshold=1):
    return all(euclidean_distance(individual, target_color) < threshold 
               for individual in population)

In [9]:
# Función para evolucionar la población durante una generación de manera generacional
def evolve_population_color(population,target_color, 
                            mutation_rate=0.4, crossover_rate=0.6):
    new_population = []

    # Cantidad de individuos que se seleccionarán para el cruce y la mutación
    crossover_count = int(len(population) * crossover_rate)
    mutation_count = len(population) - crossover_count

    # Realizar cruce
    for _ in range(crossover_count):
        parent1 = tournament_selection(population,target_color)
        parent2 = tournament_selection(population,target_color)
        child = single_point_crossover(parent1, parent2)
        new_population.append(child)

    # Realizar mutación
    for _ in range(mutation_count):
        individual = tournament_selection(population,target_color)
        mutated_individual = mutation(individual, mutation_rate)
        new_population.append(mutated_individual)

    return new_population

In [10]:
# Algoritmo Mutacion size

In [10]:
# Función para generar un tamaño de imagen aleatorio
def generate_random_size():
    return (random.randint(50, 200), random.randint(50, 200))  # Se puede ajustar el rango según sea necesario

In [11]:
# Función para generar una población inicial de tamaños de imagen aleatorios
def generate_initial_population_size(population_size):
    return [generate_random_size() for _ in range(population_size)]

In [12]:
# Función de mutación para ajustar aleatoriamente el tamaño de la imagen
def mutation_size(individual, mutation_rate=0.8):
    mutated_individual = list(individual)  # Convertir la tupla a lista para permitir la mutación
    if random.random() < mutation_rate:
        # Ajustar aleatoriamente el ancho y el alto de la imagen
        mutated_individual[0] = max(10, individual[0] + random.randint(-20, 20))
        mutated_individual[1] = max(10, individual[1] + random.randint(-20, 20))
    return tuple(mutated_individual)  # Convertir la lista de vuelta a tupla


In [13]:
# Función de cruce (crossover) de un punto
def single_point_crossover_size(parent1, parent2):
    crossover_point = random.randint(0, 1)  # Seleccionar un punto de cruce (0 para el ancho, 1 para el alto)
    child = list(parent1)
    child[crossover_point] = parent2[crossover_point]
    return tuple(child)

In [14]:
# Función para determinar si se ha alcanzado el criterio de parada (tamaño de imagen cercano al objetivo)
def has_converged_size(population, target_size, threshold=10):
    return all(abs(individual[0] - target_size[0]) < threshold 
               and abs(individual[1] - target_size[1]) < threshold 
               for individual in population)


In [15]:
# Función para seleccionar un individuo de la población utilizando selección de torneo
def tournament_selection_size(population, target_size, k=3):
    tournament_contestants = random.sample(population, k)
    return min(tournament_contestants,
               key=lambda x: abs(x[0] - target_size[0]) + 
               abs(x[1] - target_size[1]))

In [16]:
# Función para evolucionar la población durante una generación de manera generacional
def evolve_population_sizes(population, target_size, 
                            mutation_rate=0.4, crossover_rate=0.6):
    new_population = []

    # Cantidad de individuos que se seleccionarán para el cruce y la mutación
    crossover_count = int(len(population) * crossover_rate)
    mutation_count = len(population) - crossover_count

    # Realizar cruce
    for _ in range(crossover_count):
        parent1 = tournament_selection_size(population, target_size)
        parent2 = tournament_selection_size(population, target_size)
        child = single_point_crossover(parent1, parent2)
        new_population.append(child)

    # Realizar mutación
    for _ in range(mutation_count):
        individual = tournament_selection_size(population, target_size)
        mutated_individual = mutation_size(individual, mutation_rate)
        new_population.append(mutated_individual)

    return new_population

In [17]:
def crear_imagen_gato(size, color):
    # Crear una imagen del gato con el tamaño dado
    height, width = size
    image = np.ones((height, width, 3), dtype=np.uint8) * 255  # Fondo blanco

    # Dibujar la cabeza del gato
    cv2.ellipse(image, (int(width / 2), 
                        int(height / 2)), 
                (int(width / 4), 
                 int(height / 3)), 0, 0, 360, color, -1)

    # Dibujar los ojos
    cv2.circle(image, (int(width * 0.35), 
                       int(height * 0.4)), 
               int(width * 0.05), 
               (255, 255, 255), -1)
    cv2.circle(image, (int(width * 0.65), 
                       int(height * 0.4)), 
               int(width * 0.05), 
               (255, 255, 255), -1)

    # Dibujar la nariz
    cv2.ellipse(image, (int(width / 2), int(height * 0.55)), 
                (int(width * 0.1), 
                 int(height * 0.1)), 0, 0, 360, (255, 0, 0), -1)

    # Dibujar las orejas
    ear_left = np.array([[int(width * 0.2), 
                          int(height * 0.15)], 
                         [int(width * 0.3), 
                          int(height * 0.05)], 
                         [int(width * 0.4), 
                          int(height * 0.15)]])
    ear_right = np.array([[int(width * 0.6), 
                           int(height * 0.15)], 
                          [int(width * 0.7), 
                           int(height * 0.05)], 
                          [int(width * 0.8), 
                           int(height * 0.15)]])
    
    cv2.fillPoly(image, [ear_left], color)
    cv2.fillPoly(image, [ear_right], color)
    
    return image  # Devolver la imagen creada


In [20]:
# def update_cat_size_and_color(canvas, cat_index, new_size, new_color):
#     global imagenes
#     global cat_photos

#     # Crear una nueva imagen del gato con el nuevo tamaño
#     image = crear_imagen_gato(new_size,new_color)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = Image.fromarray(image)
#     photo = ImageTk.PhotoImage(image)

#     # Actualizar la lista de fotos y la imagen del gato en el lienzo
#     imagenes[cat_index] = photo
#     canvas.itemconfig(cat_photos[cat_index], image=photo)


In [21]:
# import time
# def evolve_cat_sizes_and_color(canvas, target_size, target_color,population_size=10, mutation_rate=0.9, max_generations=10000):
#     global imagenes
#     population_color = generate_initial_population(population_size)
#     population_size = generate_initial_population_size(population_size)
    

#     # Evolucionar la población y actualizar los tamaños de las imágenes de los gatos en la interfaz gráfica en tiempo real
#     generations = 0
#     while not has_converged(population_size, target_size) and not has_converged_size(population_color,target_color)  and generations < max_generations :
#         population_size = evolve_population_sizes(population_size, target_size)
#         population_color = evolve_population_color(population_color, target_color)
#         # Actualizar los tamaños de las imágenes de los gatos en la interfaz gráfica en tiempo real
#         for i, (size,color) in enumerate(zip(population_size,population_color)):
# #             update_cat_size_and_color(canvas, i, size,color)
#             update_cat_size_and_color(canvas, i, size, color)

#             canvas.update()  # Actualizar la interfaz gráfica en cada generación
#         time.sleep(0.01)  # Agregar un pequeño retraso entre cada iteración para visualizar el cambio
#         generations += 1
#     print(generations)

In [18]:

def update_cat_size_and_color(cat_index, new_size, new_color):
    global imagenes
    global cat_labels

    # Crear una nueva imagen del gato con el nuevo tamaño
    image = crear_imagen_gato(new_size, new_color)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    photo = ImageTk.PhotoImage(image)

    # Actualizar la lista de fotos y la imagen del gato en el Label
    imagenes[cat_index] = photo
    cat_labels[cat_index].config(image=photo)

In [19]:
import time

def evolve_cat_sizes_and_color(target_size, target_color, 
                               population_size=10, 
                               mutation_rate=0.9, 
                               max_generations=10000):
    global imagenes
    global cat_labels

    population_color = generate_initial_population(population_size)
    population_size = generate_initial_population_size(population_size)

    # Evolucionar la población y actualizar los tamaños de las imágenes de los gatos en la interfaz gráfica en tiempo real
    generations = 0
    update_frequency = 2  # Actualizar las imágenes cada 10 iteraciones
    update_counter = 0
    nuevo_tamano= (100,100)
    nuevo_color = [10,10,10]
    while not has_converged_size(population_size, target_size) and not has_converged(population_color, target_color) and generations < max_generations:
        population_size = evolve_population_sizes(population_size, target_size)
        population_color = evolve_population_color(population_color, target_color)

        update_counter += 1
        if update_counter >= update_frequency:
            # Actualizar los tamaños de las imágenes de los gatos en la interfaz gráfica
            for i, (size, color) in enumerate(zip(population_size, population_color)):
                update_cat_size_and_color(i, size, color)
                nuevo_tamano = size
                nuevo_color = color

            generacion_label.config(text=f"Generación: {generations}")
            ventana.update()  # Actualizar la interfaz gráfica
            update_counter = 0
            
        generations += 1
        for i, (size, color) in enumerate(zip(population_size, population_color)):
            update_cat_size_and_color(i, size, color)
            nuevo_tamano = size
            nuevo_color = color

        generacion_label.config(text=f"Generación: {generations}")
        ventana.update()  # Actualizar la interfaz gráfica
#     update_cat_size_and_color(i, nuevo_tamano,nuevo_color)
    ventana.update()
    print(generations)

In [20]:
import tkinter as tk
from tkinter import ttk
from tkinter import colorchooser
from PIL import Image, ImageTk

imagenes = []
cat_labels = []
target_size = (150, 200)
target_color = [0, 0, 255]
population_size = 10
mutation_rate = 0.8
max_generations = 10000



# Función para manejar el evento de cambio en el ComboBox
def seleccionar_tamaño(event):
    global target_size
    selected_size = tamaño_combo.get()
    if selected_size == 'Pequeño':
        target_size = (100, 150)
    elif selected_size == 'Mediano':
        target_size = (150, 200)
    else:
        target_size = (200, 250)

# Función para abrir el panel de selección de colores
def seleccionar_color():
    global target_color 
    color_code = colorchooser.askcolor(title="Seleccionar color")[1]
    if color_code:
        color_label.config(bg=color_code)

        # Convertir el color hexadecimal a RGB
        rgb = tuple(int(color_code[i:i+2], 16) for i in (1, 3, 5))
        # Convertir el color RGB a BGR
        bgr = rgb_to_bgr(rgb)
        
        # Guardar el color en la variable target_color
        target_color = bgr
        print(target_color)

def rgb_to_bgr(rgb):
    return [rgb[2], rgb[1], rgb[0]]

def aceptar():
    print(target_size)
    print(target_color)
    evolve_cat_sizes_and_color(target_size, 
                               target_color, 
                               population_size, 
                               mutation_rate, 
                               max_generations)
    pass
# Función para redimensionar la imagen de fondo
def redimensionar_imagen(event):
    global fondo_img
    fondo_img = Image.open("Fondo.png")
    fondo_img = fondo_img.resize((event.width, event.height), Image.LANCZOS)
    fondo_img = ImageTk.PhotoImage(fondo_img)
    fondo_label.config(image=fondo_img)

# Crear la ventana principal
ventana = tk.Tk()
ventana.title("Interfaz de Python")

# Cargar la imagen de fondo
fondo_img = Image.open("Fondo.png")
fondo_img = ImageTk.PhotoImage(fondo_img)

# Configurar la imagen de fondo
fondo_label = tk.Label(ventana, image=fondo_img)
fondo_label.place(x=0, y=0, relwidth=1, relheight=1)

# Vincular el evento <Configure> para redimensionar la imagen de fondo
ventana.bind("<Configure>", redimensionar_imagen)

# Combo Box para seleccionar el tamaño de la imagen
tamaño_label = tk.Label(ventana, text="Seleccionar tamaño:")
tamaño_label.place(x=50, y=50)

tamaños = ["Pequeño", "Mediano", "Grande"]
tamaño_combo = ttk.Combobox(ventana, values=tamaños)
tamaño_combo.current(0)  # Seleccionar el primer tamaño por defecto
tamaño_combo.bind("<<ComboboxSelected>>", seleccionar_tamaño)
tamaño_combo.place(x=180, y=50)

# Sección para seleccionar el color de la imagen
color_label = tk.Label(ventana, text="Seleccionar color:")
color_label.place(x=400, y=50)

color_elegido = tk.Label(ventana, bg="white", width=10, height=1)
color_elegido.place(x=500, y=50)

color_btn = tk.Button(ventana, text="Elegir color", command=seleccionar_color)
color_btn.place(x=600, y=50)

# Botón de aceptar
aceptar_btn = tk.Button(ventana, text="Aceptar", command=aceptar)
aceptar_btn.place(x=800, y=50)
# Crear una etiqueta para mostrar el valor de la generación
generacion_label = tk.Label(ventana, text="Generación: 0")
generacion_label.place(x=900, y=100)

# Crear las imágenes de los gatos y colocarlas en la interfaz
x, y = 100, 150
size = generate_initial_population_size(10)
color = generate_initial_population(10)
# print(size)
# print(color)
for i, (size, color) in enumerate(zip(size, color)):
#     print(size)
#     print(color)
    image = crear_imagen_gato(size, color)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    photo = ImageTk.PhotoImage(image)

    # Agregar la imagen a la lista para mantener la referencia
    imagenes.append(photo)

    # Crear un Label para la imagen del gato y colocarlo en la interfaz
    cat_label = tk.Label(ventana, image=photo)
    cat_label.place(x=x, y=y)
    cat_labels.append(cat_label)

    # Ajustar las coordenadas para la próxima imagen
    x += 300  # Aumentar la coordenada y para separar las imágenes
    if x >= 1100:
        x = 100
        y += 250


ventana.mainloop()

In [25]:
# RGB -> BGR

SyntaxError: invalid syntax (27088084.py, line 1)

In [29]:
# import tkinter as tk
# from tkinter import ttk
# from tkinter import colorchooser
# from PIL import Image, ImageTk

# imagenes = []
#     # Lista para almacenar los objetos PhotoImage de los gatos
# cat_photos = []
# target_size = (150, 200)
# target_color = [ 0, 0,255]
# population_size = 10
# mutation_rate = 0.8
# max_generations = 10000
# #
# def aceptar():
#     # Aquí iría la lógica para procesar la selección de tamaño y color de la imagen
#     print(target_size)
#     print(target_color)
#     evolve_cat_sizes_and_color(canvas, target_size, target_color, population_size, mutation_rate, max_generations)
#     pass

# # Función para manejar el evento de cambio en el ComboBox
# def seleccionar_tamaño(event):
#     selected_size = tamaño_combo.get()
#     if(selected_size == 'Pequeño'):
# #         print("Tamaño seleccionado: 100 x 150")
#         target_size = (100, 150)
#     elif(selected_size == 'Mediano'):
# #         print("Tamaño seleccionado: 150 x 200")
#         target_size = (150,200)
#     else: 
# #         print("Tamaño seleccionado: 200 x 250")
#         target_size = (200,250)
    

# # Función para abrir el panel de selección de colores
# def seleccionar_color():
#     color_code = colorchooser.askcolor(title="Seleccionar color")[1]
#     if color_code:
#         color_label.config(bg=color_code)
#         print("Color seleccionado en formato hexadecimal:", color_code)

#         # Convertir el color hexadecimal a RGB
#         rgb = tuple(int(color_code[i:i+2], 16) for i in (1, 3, 5))
        
#         # Guardar el color en la variable target_color
#         target_color = list(rgb)
# #         print("Color seleccionado en formato RGB:", target_color)

# # Función para redimensionar la imagen de fondo
# def redimensionar_imagen(event):
#     global fondo_img
#     fondo_img = Image.open("Fondo.png")
#     fondo_img = fondo_img.resize((event.width, event.height), Image.LANCZOS)
#     fondo_img = ImageTk.PhotoImage(fondo_img)
#     fondo_label.config(image=fondo_img)

# # Crear la ventana principal
# ventana = tk.Tk()
# ventana.title("Interfaz de Python")

# # Cargar la imagen de fondo
# fondo_img = Image.open("Fondo.png")
# fondo_img = ImageTk.PhotoImage(fondo_img)

# # Configurar la imagen de fondo
# fondo_label = tk.Label(ventana, image=fondo_img)
# fondo_label.place(x=0, y=0, relwidth=1, relheight=1)

# # Vincular el evento <Configure> para redimensionar la imagen de fondo
# ventana.bind("<Configure>", redimensionar_imagen)


# # Combo Box para seleccionar el tamaño de la imagen
# tamaño_label = tk.Label(ventana, text="Seleccionar tamaño:")
# tamaño_label.place(x=50, y=50)

# tamaños = ["Pequeño", "Mediano", "Grande"]
# tamaño_combo = ttk.Combobox(ventana, values=tamaños)
# tamaño_combo.current(0)  # Seleccionar el primer tamaño por defecto
# tamaño_combo.bind("<<ComboboxSelected>>", seleccionar_tamaño)
# tamaño_combo.place(x=180, y=50)

# # Sección para seleccionar el color de la imagen
# color_label = tk.Label(ventana, text="Seleccionar color:")
# color_label.place(x=50, y=100)

# color_elegido = tk.Label(ventana, bg="white", width=10, height=1)
# color_elegido.place(x=180, y=100)

# color_btn = tk.Button(ventana, text="Elegir color", command=seleccionar_color)
# color_btn.place(x=300, y=100)

# # Botón de aceptar
# aceptar_btn = tk.Button(ventana, text="Aceptar", command=aceptar)
# aceptar_btn.place(x=150, y=150)

# # Configurar el tamaño de la ventana
# ventana.geometry("1400x1200")

#     # Crear un lienzo para colocar las imágenes de los gatos
# canvas = tk.Canvas(ventana, width=1000, height=1200)
# canvas.pack()
# x, y = 100, 150
# size = generate_initial_population_size(10)
# color = generate_initial_population(10)
# print(size)
# print(color)
# for i,(size,color) in enumerate(zip(size,color)):
#     print(size)
#     print(color)
#         # Crear una imagen de gato inicial con un tamaño aleatorio
# #         size = generate_random_size()
#     image = crear_imagen_gato(size,color)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = Image.fromarray(image)
#     photo = ImageTk.PhotoImage(image)

#         # Agregar la imagen a la lista para mantener la referencia
#     imagenes.append(photo)

#         # Colocar la imagen en el lienzo y almacenar el objeto PhotoImage
#     cat_photo = canvas.create_image(x, y, image=photo)
#     cat_photos.append(cat_photo)

#         # Ajustar las coordenadas para la próxima imagen
#     x += 300  # Aumentar la coordenada y para separar las imágenes
#     if x >= 800:
#         x = 100
#         y += 200

# # Iniciar el bucle de eventos
# ventana.mainloop()

[(68, 92), (124, 94), (128, 73), (53, 194), (83, 150), (135, 57), (143, 114), (88, 124), (81, 194), (182, 158)]
[[1, 118, 163], [179, 250, 219], [109, 50, 60], [248, 163, 253], [248, 19, 236], [32, 7, 201], [2, 133, 57], [138, 121, 57], [171, 254, 7], [38, 192, 5]]
(68, 92)
[1, 118, 163]
(124, 94)
[179, 250, 219]
(128, 73)
[109, 50, 60]
(53, 194)
[248, 163, 253]
(83, 150)
[248, 19, 236]
(135, 57)
[32, 7, 201]
(143, 114)
[2, 133, 57]
(88, 124)
[138, 121, 57]
(81, 194)
[171, 254, 7]
(182, 158)
[38, 192, 5]
(150, 200)
[0, 0, 255]
47
